### 1. import libraries

In [1]:
# from AhpAnpLib import *
from AhpAnpLib import inputs_AHPLib as input
from AhpAnpLib import structs_AHPLib as str
from AhpAnpLib import calcs_AHPLib as calc
from AhpAnpLib import ratings_AHPLib as rate

### 2. Set up AHP model structure

In [2]:
#create model
lunchModel=str.Model("Where to order lunch")

In [3]:
#create goal and criteria nodes
# goal
goal_node=str.Node("GoalNode",0)
# criteria
quality=str.Node("1Quality",1)
price=str.Node("2Price",2)
menu=str.Node("3Menu",3)
speed=str.Node("4Speed",4)

# alternative nodes can be created here or later
#alt1=str.Node("1Primanti",5)
#alt2=str.Node("2Panera",6)
#alt3=str.Node("3Piada",7)

In [4]:
#create clusters
cluster0=str.Cluster("1Goal",0)
cluster1=str.Cluster("2Criteria",1)

#add nodes to clusters
cluster0.addNode2Cluster(goal_node) 

cluster1.addNode2Cluster(menu)
cluster1.addNode2Cluster(price)
cluster1.addNode2Cluster(quality)
cluster1.addNode2Cluster(speed)

In [5]:
#add goal and criteria clusters to the model 
lunchModel.addCluster2Model(cluster0)
lunchModel.addCluster2Model(cluster1)

In [6]:
#set up node connections from Goal Node to all the nodes of the 2Criteria cluster
#we don't add connections to alternatives in the ratings model
lunchModel.addNodeConnectionFromNodeToAllNodesOfCluster("GoalNode","2Criteria")

# Print out model to validate the structure
lunchModel.printStruct()

_________________________MODEL STRUCTURE_________________________
Name: Where to order lunch
Type: pairwise

____________________________NODES_______________________________
GoalNode order: 0
1Quality order: 1
2Price order: 2
3Menu order: 3
4Speed order: 4
____________________________CLUSTERS____________________________
1Goal order: 0
2Criteria order: 1
_________________________NODE CONNECTIONS___________________________
Connections from node GoalNode order: 0
 to: 3Menu order: 3
 to: 2Price order: 2
 to: 1Quality order: 1
 to: 4Speed order: 4
No connections from 1Quality
No connections from 2Price
No connections from 3Menu
No connections from 4Speed
_________________________CLUSTER CONNECTIONS__________________
Connection(s) from cluster 1Goal order: 0
 to: 2Criteria order: 1
No connections from 2Criteria


### 3. Set up ratings model

#### Step 1 Set the model to be a ratings model

In [7]:
# set the model to be a ratings model using the setModelTypeRatings command
lunchModel.setModelTypeRatings()

#### Step 2 Add Criteria to the model

In [8]:
# select all criteria from bottom level criteria (if you have subcriteria, then no need to add their parent criteria)
# we can use either rateModel.addCriteriaByVar command to add criteria using variable name
# or we can use rateModel.addCriteriaByName command to add criteria using names of criteria
lunchModel.rateModel.addCriteriaByVar(quality,price,menu)
lunchModel.rateModel.addCriteriaByName("4Speed")

#### Step 3 Add Alternatives to the model

In [9]:
# add ratings altenatives
lunchModel.rateModel.addAlternativesByName("1Primanti","2Panera","3Piada")

#### Step 4 Create/Import scales and asisgn scales to ratings criteria

In [11]:

# Create scales to use for the evaluation of the alternatives with respect to the selected criteria
scaleEtoP=input.readRatScaleRPCfile("ExcellentToPoor","ExcellentToPoor.rcp")
# add the scale to the model
lunchModel.rateModel.addScaleByVar(scaleEtoP)
# assign the scale to criteria
# we can assign the same scale to multiple criteria
lunchModel.rateModel.assignScale2CriterionByName("1Quality","ExcellentToPoor")
lunchModel.rateModel.assignScale2CriterionByName("3Menu","ExcellentToPoor")

# scale for price
scalePrice=input.readRatScaleRPCfile("PriceScale","HitoLo.rcp")
lunchModel.rateModel.addScaleByVar(scalePrice)
lunchModel.rateModel.assignScale2CriterionByName("2Price","PriceScale")

# scale for speed
scaleSpeed=rate.RatScale("SpeedScale")
# we can assign values to the scale or we can leave them as empty 
# e.g. scaleSpeed.defineScaleByValue(None,False,"Fast","Average","Slow")
scaleSpeed.defineScaleByValue(None,False,["Fast",1],["Average",0.5],["Slow",0.2])
lunchModel.rateModel.addScaleByVar(scaleSpeed)
lunchModel.rateModel.assignScale2CriterionByName("4Speed","SpeedScale")

#### Step 5 Make judgment of the AHP criteria to get priorities

In [12]:
#Export Excel questionnaire for criteria
input.export4ExcelQuestFull(lunchModel,"lunchModel_Ratings_Criteria_empty.xlsx",True)
#Import Criteria questionnaire to calculate priorities 
input.importFromExcel(lunchModel,"lunchModel_Ratings_Criteria_filledin.xlsx","pairwise_comp",False)
calc.calcAHPMatricesSave2File(lunchModel,"lunchModel_Ratings_Criteria_filledin.xlsx","LunchModel_Ratings_Criteria_initialresults.xlsx",True,False,True)

#### Step 6 Make Ratings in ratings table

In [13]:
#Export Excel questionnaire for ratings invluding ratings scale and ratings table
input.export4ExcelRatingsSetup(lunchModel,"LunchModel_Ratings_Table_empty.xlsx") 

load


In [14]:
#Import ratings table, calculate and get the final ratings results
input.calcExcelRatings(lunchModel,"LunchModel_Ratings_Table_filledIn.xlsx","carModel_Ratings_Results.xlsx",True)

row: 0--for criterion 1Quality
scale ExcellentToPoor
[[1.000 2.000 5.000 7.000 9.000]
 [0.500 1.000 4.000 5.000 8.000]
 [0.200 0.250 1.000 4.000 7.000]
 [0.143 0.200 0.250 1.000 3.000]
 [0.111 0.125 0.143 0.333 1.000]]
row: 10--for criterion 2Price
scale PriceScale
[[1.000 0.347 0.080]
 [2.884 1.000 0.231]
 [12.480 4.327 1.000]]
row: 18--for criterion 3Menu
scale ExcellentToPoor_cp
[[1.000 2.000 5.000 7.000 9.000]
 [0.500 1.000 4.000 5.000 8.000]
 [0.200 0.250 1.000 4.000 7.000]
 [0.143 0.200 0.250 1.000 3.000]
 [0.111 0.125 0.143 0.333 1.000]]
row: 28--for criterion 4Speed
scale SpeedScale
[[1.000 2.000 5.000]
 [0.500 1.000 2.500]
 [0.200 0.400 1.000]]
Checking:Above Average
??=Excellent
??=Above Average
Found in pos 1
Scale: [[1.000 2.000 5.000 7.000 9.000]
 [0.500 1.000 4.000 5.000 8.000]
 [0.200 0.250 1.000 4.000 7.000]
 [0.143 0.200 0.250 1.000 3.000]
 [0.111 0.125 0.143 0.333 1.000]]
 Priorities [0.463 0.307 0.142 0.058 0.030] 
 Ideal [1.0, 0.6643259866829985, 0.30614159048892337

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
